In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import pandas as pd

# Get the absolute path to the modules directory
module_path = os.path.abspath(os.path.join("..", "project_s"))
if module_path not in sys.path:
    sys.path.append(module_path)


## Run for one week

In [ ]:
from graph import build_graph
from graph_state import GraphState

from config import (
    DATA_JOURNAL_TXT_FILE,
    DATA_JOURNAL_PICKLE,
    DATA_WEEKLY_PERCEPTION_JSON_FILE,
    DATA_WEEKLY_NOTES_PICKLE,
    DATA_SCREEN_TIME_PICKLE,
    
    TWILIO_ACCOUNT_SID,
    TWILIO_AUTH_TOKEN,
    FROM_NUMBER,
    TO_NUMBER,
    OPENAI_API_KEY
)

initial_state = GraphState(
    journal_text=open(DATA_JOURNAL_TXT_FILE).read(),
    weekly_perception_text=open(DATA_WEEKLY_PERCEPTION_JSON_FILE).read(),
    weekly_notes_df=pd.read_pickle("../data/weekly_notes/week_Jun_23–29.pickle"),
    screen_time_df=pd.read_pickle("../data/screen_time/week_Jun_23–29.pickle"),
    calendar_df=pd.read_pickle("../data/calendar/week_Jun_23–29.pickle"),
    openai_key=OPENAI_API_KEY,
    twilio_sid=TWILIO_ACCOUNT_SID,
    twilio_token=TWILIO_AUTH_TOKEN,
    twilio_from="whatsapp:"+FROM_NUMBER,
    twilio_to="whatsapp:"+TO_NUMBER
)

graph = build_graph()
final_state = graph.invoke(initial_state)



## Visualize Graph

In [ ]:
def export_mermaid_manual(filename="reflective_graph.mmd"):
    mermaid_text = """graph TD
    perceive --> reflect
    reflect --> whisper
    whisper --> END
    """

    with open(filename, "w") as f:
        f.write(mermaid_text)

export_mermaid_manual()

## Simulate run for all weeks

In [ ]:
import os
import pandas as pd
from graph import build_graph
from graph_state import GraphState
from perception import perception_node
from config import (
    DATA_JOURNAL_TXT_FILE,
    DATA_WEEKLY_NOTES_DIR,
    DATA_SCREEN_TIME_DIR,
    DATA_CALENDAR_DIR,
    OPENAI_API_KEY,
    TWILIO_ACCOUNT_SID,
    TWILIO_AUTH_TOKEN,
    FROM_NUMBER,
    TO_NUMBER,
)

# Load static journal text
journal_text = open(DATA_JOURNAL_TXT_FILE).read()

# Compile the graph once
graph = build_graph()

# Loop through all weeks
for filename in sorted(os.listdir(DATA_WEEKLY_NOTES_DIR)):
    if not filename.endswith(".pickle"):
        continue

    # Extract base name (e.g. week_Jun_23–29)
    week_id = filename.replace(".pickle", "")
    print(f"📆 Processing {week_id}")

    # Build file paths
    notes_path = os.path.join(DATA_WEEKLY_NOTES_DIR, filename)
    screen_path = os.path.join(DATA_SCREEN_TIME_DIR, f"{week_id}.pickle")
    calendar_path = os.path.join(DATA_CALENDAR_DIR, f"{week_id}.pickle")

    # Load data
    weekly_notes_df = pd.read_pickle(notes_path)
    screen_time_df = pd.read_pickle(screen_path)
    calendar_df = pd.read_pickle(calendar_path)

    # Derive perception - currently not used in the agentic design/socratic node, but prepared for a future extension.
    perception_df = perception_node(weekly_notes_df, screen_time_df, calendar_df)
    weekly_perception_text = perception_df.to_string(index=False)

    # Build state
    state = GraphState(
        journal_text=journal_text,
        weekly_perception_text=weekly_perception_text,
        weekly_notes_df=weekly_notes_df,
        screen_time_df=screen_time_df,
        calendar_df=calendar_df,
        openai_key=OPENAI_API_KEY,
        twilio_sid=TWILIO_ACCOUNT_SID,
        twilio_token=TWILIO_AUTH_TOKEN,
        twilio_from="whatsapp:" + FROM_NUMBER,
        twilio_to="whatsapp:" + TO_NUMBER
    )

    # Run the graph
    # final_state = graph.invoke(state)
    final_state_raw = graph.invoke(state)
    final_state = GraphState(**final_state_raw)
    
    # Print preview
    whisper_meta = final_state.whisper_status or {}
    preview = whisper_meta.get("preview", "<no preview>")
    print(f"✅ Message sent for {week_id}:")
    print(preview)
    print("-" * 40)
